In [4]:
# -*- coding: utf-8 -*-
"""
# 📊 SRC TRADING - Japanese Candlestick Pattern Detector
---
This Python notebook is designed to help traders and investors automatically identify classic Japanese candlestick patterns in stock or cryptocurrency price data.
Japanese candlesticks are a way of representing the price movements of an asset over a period of time, showing the open, close, high, and low prices. The patterns formed by one or more candles are used in technical analysis to predict potential future market movements.

Below, follow the steps to:
1. Install the necessary libraries.
2. Download data for a specific asset.
3. Visualize the candlestick chart.
4. Detect and interpret candlestick patterns.
"""

# In[0]:
### Visual Header
#
# Loads and displays the SRC TRADING logo
from IPython.display import display, Image, clear_output
import ipywidgets as widgets

header_html = """
<style>
    .header-container {
        display: flex;
        align-items: center;
        background-color: #1a1a1a;
        padding: 20px;
        border-radius: 10px;
        margin-bottom: 20px;
    }
    .header-text h1 {
        color: #fff;
        font-family: sans-serif;
        font-size: 2.5em;
        margin: 0;
        padding-left: 20px;
    }
    .header-text p {
        color: #bbb;
        font-family: sans-serif;
        font-size: 1em;
        margin: 5px 0 0 20px;
    }
</style>
<div class="header-container">
    <img src="https://placehold.co/100x100/1a1a1a/fff?text=SRC" alt="SRC TRADING Logo" style="border-radius: 8px;">
    <div class="header-text">
        <h1>SRC TRADING</h1>
        <p>Japanese Candlestick Pattern Detector</p>
    </div>
</div>
"""
display(widgets.HTML(header_html))

# In[1]:
### 1. Library Installation
#
# Run this cell to install all necessary dependencies.
# The libraries we will use are:
# - `yfinance`: To download financial data from Yahoo Finance.
# - `plotly`: To create interactive charts.
# - `ipywidgets`: To create the interactive user interface in the notebook.
# - `pandas`: For handling data in DataFrames.
# - `openpyxl`: To export to Excel files.
# - `fpdf`: To generate PDF files.

!pip install yfinance plotly pandas ipywidgets openpyxl fpdf --quiet

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, FileLink
import numpy as np
import plotly.graph_objects as go
import io
from fpdf import FPDF

# In[2]:
### 2. Data Download
#
# Use the controls below to enter the stock or cryptocurrency ticker and the date range to download the data.
# Examples of tickers: `AAPL` (Apple), `TSLA` (Tesla), `BTC-USD` (Bitcoin).

# Widget configuration for the user interface
style = {'description_width': 'initial'}
ticker_input = widgets.Text(
    value='BTC-USD',
    description='Asset Ticker:',
    placeholder='Ex: AAPL, TSLA, BTC-USD',
    style=style
)
start_date_picker = widgets.DatePicker(
    description='Start Date:',
    value=(datetime.now() - timedelta(days=180)).date(),
    disabled=False,
    style=style
)
end_date_picker = widgets.DatePicker(
    description='End Date:',
    value=datetime.now().date(),
    disabled=False,
    style=style
)
# -- START OF CHANGE: Timeframe selector
timeframe_selector = widgets.Dropdown(
    options=['1d', '1h', '15m', '5m', '1m'],
    value='1d',
    description='Timeframe:',
    style=style
)
# -- END OF CHANGE --
download_button = widgets.Button(
    description='Download Data',
    button_style='primary',
    icon='download'
)
output = widgets.Output()

def download_data(b):
    with output:
        clear_output(wait=True)
        ticker = ticker_input.value
        start_date = start_date_picker.value
        end_date = end_date_picker.value
        timeframe = timeframe_selector.value

        if start_date >= end_date:
            print("Error! The start date must be before the end date.")
            return

        print(f"Downloading data for {ticker} from {start_date} to {end_date} with a {timeframe} interval...")
        try:
            global df
            df = yf.download(ticker, start=start_date, end=end_date, interval=timeframe)
            if df.empty:
                print(f"Error! No data found for ticker '{ticker}' in the selected date range. Please verify the ticker and try again.")
            else:
                print("Download complete! First 5 rows of the DataFrame:")
                display(df.head())
        except Exception as e:
            print(f"An error occurred during download: {e}")

download_button.on_click(download_data)

# Show the widgets in the notebook
print("Enter the ticker and date range:")
# -- START OF CHANGE: Display the new selector
display(ticker_input, start_date_picker, end_date_picker, timeframe_selector, download_button, output)
# -- END OF CHANGE --

# In[3]:
### 3. Candlestick Visualization
#
# This function plots the interactive candlestick and volume chart using Plotly.

def graficar_velas(df, patterns_df=None):
    """
    Creates an interactive candlestick chart with volume and patterns using Plotly.
    """
    if df.empty:
        print("The DataFrame is empty. Please download the data first.")
        return

    # Create the main figure
    fig = go.Figure()

    # Add the candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='Candles',
            increasing_line_color='#1b5e20',  # Green
            decreasing_line_color='#d32f2f'   # Red
        )
    )

    # Add patterns as point traces
    if patterns_df is not None and not patterns_df.empty:
        unique_patterns = patterns_df['pattern'].unique()
        for pattern_name in unique_patterns:
            pat_data = patterns_df[patterns_df['pattern'] == pattern_name]

            # Use a different symbol for patterns to distinguish them
            marker_symbol = 'triangle-up'
            if 'Hammer' in pattern_name or 'Doji' in pattern_name:
                marker_symbol = 'circle'
            elif 'Star' in pattern_name:
                marker_symbol = 'star'

            fig.add_trace(
                go.Scatter(
                    x=pat_data['date'],
                    y=pat_data['value'],
                    mode='markers',
                    marker=dict(
                        symbol=marker_symbol,
                        size=10,
                        color=pat_data.iloc[0]['color'],
                        line=dict(width=1, color='black')
                    ),
                    name=pattern_name,
                    hoverinfo='text',
                    hovertext=[f'Pattern: {p}' for p in pat_data['pattern']]
                )
            )

    # Add the volume chart
    fig.add_trace(
        go.Bar(
            x=df.index,
            y=df['Volume'],
            name='Volume',
            marker_color='gray',
            yaxis='y2' # Assign to a second Y-axis
        )
    )

    # Update the layout for the two charts
    fig.update_layout(
        title=f'Candlestick Chart for {ticker_input.value}',
        xaxis_rangeslider_visible=False, # Hide the range slider control
        xaxis_title='Date',
        yaxis_title='Price',
        yaxis2=dict(
            title='Volume',
            overlaying='y',
            side='right',
            showgrid=False
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        template='plotly_dark' # A dark theme for better visualization
    )

    print("Generating interactive chart...")
    fig.show()

# In[4]:
### 4. Pattern Detection
#
# Here the logic is implemented to detect classic candlestick patterns.
# Each function checks a specific candle and its predecessors according to the pattern's definition.

def is_doji(open_price, close_price, high, low):
    """
    A Doji is formed when the open and close prices are nearly identical.
    This indicates market indecision.
    """
    return np.isclose(open_price, close_price, rtol=0.01) and (high - low) > (high - close_price) * 1.5 and (high - low) > (open_price - low) * 1.5

def is_hammer(open_price, close_price, high, low):
    """
    The Hammer is a bullish pattern. It has a small body at the top of the range
    and a long lower wick, indicating that buyers rejected the low price.
    """
    body_size = abs(close_price - open_price)
    lower_shadow = min(open_price, close_price) - low
    upper_shadow = high - max(open_price, close_price)
    return (lower_shadow >= 2 * body_size) and (upper_shadow < body_size)

def is_shooting_star(open_price, close_price, high, low):
    """
    The Shooting Star is a bearish pattern. It has a small body at the
    bottom of the range and a long upper wick, indicating that sellers rejected the high price.
    """
    body_size = abs(close_price - open_price)
    upper_shadow = high - max(open_price, close_price)
    lower_shadow = min(open_price, close_price) - low
    return (upper_shadow >= 2 * body_size) and (lower_shadow < body_size)

def is_engulfing(prev_open, prev_close, open_price, close_price):
    """
    The Engulfing pattern is formed when a large candle completely
    engulfs the body of the previous candle.
    """
    bullish = (prev_close > prev_open) and (close_price > open_price) and (open_price < prev_close) and (close_price > prev_close) and ((open_price < prev_open) or (close_price > prev_close))
    bearish = (prev_close < prev_open) and (close_price < open_price) and (open_price > prev_close) and (close_price < prev_close) and ((open_price > prev_open) or (close_price < prev_close))
    return bullish, bearish

def is_harami(prev_open, prev_close, open_price, close_price):
    """
    The Harami pattern is formed when a small candle is contained within
    the body of the previous candle.
    """
    bullish = (prev_close < prev_open) and (close_price > open_price) and (open_price > prev_close) and (close_price < prev_open)
    bearish = (prev_close > prev_open) and (close_price < open_price) and (open_price < prev_close) and (close_price > prev_open)
    return bullish, bearish

def is_spinning_top(open_price, close_price, high, low):
    """
    A Spinning Top has a small body and long wicks on both sides.
    It indicates indecision.
    """
    body_size = abs(close_price - open_price)
    full_range = high - low
    upper_shadow = high - max(open_price, close_price)
    lower_shadow = min(open_price, close_price) - low
    return (body_size < 0.3 * full_range) and (upper_shadow > body_size) and (lower_shadow > body_size)

def is_dragonfly_doji(open_price, close_price, high, low):
    """
    A Dragonfly Doji has a long lower wick and the body at the
    top. It is a bullish pattern.
    """
    return np.isclose(open_price, close_price, rtol=0.01) and np.isclose(open_price, high, rtol=0.01) and (open_price - low) > (high - open_price) * 3

def is_gravestone_doji(open_price, close_price, high, low):
    """
    A Gravestone Doji has a long upper wick and the body at the
    bottom. It is a bearish pattern.
    """
    return np.isclose(open_price, close_price, rtol=0.01) and np.isclose(open_price, low, rtol=0.01) and (high - open_price) > (open_price - low) * 3

def is_marubozu(open_price, close_price, high, low):
    """
    A Marubozu is a candle with a very large body and no wicks.
    It indicates a strong movement in one direction.
    """
    full_range = high - low
    body_size = abs(close_price - open_price)
    # Bullish Marubozu
    bullish = (close_price > open_price) and (body_size > 0.9 * full_range) and (low == open_price) and (high == close_price)
    # Bearish Marubozu
    bearish = (close_price < open_price) and (body_size > 0.9 * full_range) and (low == close_price) and (high == open_price)
    return bullish, bearish


def detect_patterns(df):
    """
    Main function to iterate over the DataFrame and detect all patterns.
    Returns a DataFrame with the dates and the patterns found.
    """
    detected_patterns = []

    # Ensure the DataFrame has a date index
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # Convert to numpy arrays for better performance
    opens = df['Open'].values
    highs = df['High'].values
    lows = df['Low'].values
    closes = df['Close'].values

    for i in range(1, len(df)):
        open_price = opens[i]
        close_price = closes[i]
        high = highs[i]
        low = lows[i]

        prev_open = opens[i-1]
        prev_close = closes[i-1]

        # 1. Single-candle patterns
        if is_doji(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Doji', 'value': close_price, 'color': 'yellow'})

        if is_hammer(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Hammer', 'value': low, 'color': 'blue'})

        if is_shooting_star(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Shooting Star', 'value': high, 'color': 'purple'})

        is_marubozu_bullish, is_marubozu_bearish = is_marubozu(open_price, close_price, high, low)
        if is_marubozu_bullish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Bullish Marubozu', 'value': close_price, 'color': 'green'})
        if is_marubozu_bearish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Bearish Marubozu', 'value': close_price, 'color': 'red'})

        if is_dragonfly_doji(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Dragonfly Doji', 'value': low, 'color': 'cyan'})

        if is_gravestone_doji(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Gravestone Doji', 'value': high, 'color': 'magenta'})

        # 2. Two-candle patterns
        is_engulfing_bullish, is_engulfing_bearish = is_engulfing(prev_open, prev_close, open_price, close_price)
        if is_engulfing_bullish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Bullish Engulfing', 'value': low, 'color': 'darkgreen'})
        if is_engulfing_bearish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Bearish Engulfing', 'value': high, 'color': 'darkred'})

        is_harami_bullish, is_harami_bearish = is_harami(prev_open, prev_close, open_price, close_price)
        if is_harami_bullish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Bullish Harami', 'value': low, 'color': 'olive'})
        if is_harami_bearish:
            detected_patterns.append({'date': df.index[i], 'pattern': 'Bearish Harami', 'value': high, 'color': 'maroon'})

        if is_spinning_top(open_price, close_price, high, low):
            detected_patterns.append({'date': df.index[i], 'pattern': 'Spinning Top', 'value': close_price, 'color': 'orange'})

    return pd.DataFrame(detected_patterns)


# In[5]:
### 5. Results and Interpretation
#
# This section shows the detected patterns and allows you to visualize them on the chart.

def show_results():
    """
    Function to display the detection results and the user interface.
    """
    if 'df' not in globals() or df.empty:
        print("No data loaded. Please download the data first in section 2.")
        return

    if len(df) < 2:
        print("Not enough data to detect patterns. At least two candles are needed.")
        return

    global patterns_df
    patterns_df = detect_patterns(df.copy())

    print("\n### Detected Patterns:")
    if patterns_df.empty:
        print("No patterns were found in the selected date range.")
        return

    # Show a table with the last 10 detections
    print("Last 10 pattern detections:")
    display(patterns_df.tail(10))

    # Buttons to download results
    download_csv_button = widgets.Button(
        description='Download CSV',
        button_style='info',
        icon='save'
    )
    download_excel_button = widgets.Button(
        description='Download Excel',
        button_style='info',
        icon='save'
    )
    download_pdf_button = widgets.Button(
        description='Download PDF',
        button_style='info',
        icon='save'
    )

    def download_csv(b):
        csv_output = patterns_df.to_csv(index=False)
        display(FileLink('patterns_detected.csv', data=csv_output, result_html_prefix="Click to download the file:"))

    def download_excel(b):
        patterns_df.to_excel('patterns_detected.xlsx', index=False)
        display(FileLink('patterns_detected.xlsx', result_html_prefix="Click to download the file:"))

    def download_pdf(b):
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt="Japanese Candlestick Pattern Report", ln=True, align='C')
        pdf.ln(10)

        # Add the results table
        pdf.set_font("Arial", size=10)
        # Header
        for col in patterns_df.columns:
            pdf.cell(40, 7, col, 1, 0, 'C')
        pdf.ln()
        # Data
        for index, row in patterns_df.iterrows():
            pdf.cell(40, 7, str(row['date'].strftime('%Y-%m-%d')), 1, 0)
            pdf.cell(40, 7, row['pattern'], 1, 0)
            pdf.cell(40, 7, str(round(row['value'], 2)), 1, 0)
            pdf.cell(40, 7, row['color'], 1, 0)
            pdf.ln()

        pdf_output = pdf.output(dest='S').encode('latin-1')
        with open('patterns_detected.pdf', 'wb') as f:
            f.write(pdf_output)
        display(FileLink('patterns_detected.pdf', result_html_prefix="Click to download the file:"))

    download_csv_button.on_click(download_csv)
    download_excel_button.on_click(download_excel)
    download_pdf_button.on_click(download_pdf)

    print("\n---")
    print("Select the patterns you want to see on the chart:")

    # Widgets to filter patterns on the chart
    all_patterns = patterns_df['pattern'].unique()
    checkboxes = [widgets.Checkbox(description=p, value=True) for p in all_patterns]
    checkbox_container = widgets.VBox(checkboxes)
    display(checkbox_container)

    plot_button = widgets.Button(
        description='Generate Chart with Patterns',
        button_style='success',
        icon='bar-chart'
    )

    def plot_with_patterns(b):
        with output:
            clear_output(wait=True)
            selected_patterns = [cb.description for cb in checkboxes if cb.value]
            if not selected_patterns:
                print("Please select at least one pattern to plot.")
                return

            # Filter the patterns for the chart
            filtered_patterns = patterns_df[patterns_df['pattern'].isin(selected_patterns)]

            # Call the plotting function with the filtered patterns
            graficar_velas(df, patterns_df=filtered_patterns)

    plot_button.on_click(plot_with_patterns)

    # Show the user interface with the download buttons
    display(plot_button, widgets.HBox([download_csv_button, download_excel_button, download_pdf_button]), output)

# Run the function to show the results interface
# You must have downloaded the data in section 2 before running this
if 'df' in locals() and not df.empty:
    show_results()


# In[6]:
### 6. Pattern Backtest
#
# Simulates long positions (buys) based on the appearance of a specific pattern.
#
# This backtest is "lightweight" and works as follows:
# - A long position is assumed for each time the selected pattern appears.
# - The entry is made at the closing price of the candle where the pattern is detected.
# - The exit is made at the closing price of the `Holding Period` after the entry.
#
# The backtest does not consider commissions, slippage, or other real trading considerations.

def backtest_patron(df_original, patterns_df, selected_pattern, holding_period):
    """
    Performs a simple backtest for a specific candlestick pattern.
    """
    # Filter patterns for the backtest
    trades = patterns_df[patterns_df['pattern'] == selected_pattern]

    if trades.empty:
        return {'total_trades': 0, 'winning_trades': 0, 'total_return': 0, 'max_drawdown': 0}

    capital_curve = [1000] # We start with a base capital of 1000
    profits = []

    # Ensure the original candlestick data DataFrame is sorted
    df_original = df_original.sort_index()

    for date_found in trades['date']:
        try:
            # Find the index of the pattern in the original DataFrame
            entry_idx = df_original.index.get_loc(date_found)
            exit_idx = entry_idx + holding_period

            # Ensure the exit trade is within the data
            if exit_idx < len(df_original):
                entry_price = df_original.iloc[entry_idx]['Close']
                exit_price = df_original.iloc[exit_idx]['Close']

                # Calculate the trade return
                trade_return = (exit_price - entry_price) / entry_price

                # Update metrics
                profits.append(trade_return)

                # Update capital to calculate drawdown
                last_capital = capital_curve[-1]
                new_capital = last_capital * (1 + trade_return)
                capital_curve.append(new_capital)

        except KeyError:
            # In case the date is not found in the data DataFrame
            continue

    # Calculate final metrics
    total_trades = len(profits)
    winning_trades = sum(1 for p in profits if p > 0)

    total_return = (capital_curve[-1] - capital_curve[0]) / capital_curve[0] if len(capital_curve) > 1 else 0

    # Calculate maximum drawdown
    capital_series = pd.Series(capital_curve)
    rolling_max = capital_series.expanding(min_periods=1).max()
    drawdowns = (capital_series / rolling_max) - 1
    max_drawdown = abs(drawdowns.min()) if not drawdowns.empty else 0

    return {
        'total_trades': total_trades,
        'winning_trades': winning_trades,
        'total_return': total_return,
        'max_drawdown': max_drawdown
    }

# Widgets for the backtest
backtest_output = widgets.Output()

def run_backtest(b):
    with backtest_output:
        clear_output(wait=True)
        if 'df' not in globals() or df.empty or 'patterns_df' not in globals() or patterns_df.empty:
            print("Please download the data and detect patterns first in the previous sections.")
            return

        selected_pattern = pattern_dropdown.value
        hold_period = hold_period_input.value

        if not selected_pattern:
            print("Please select a pattern to backtest.")
            return

        print(f"Running backtest for pattern '{selected_pattern}' with a holding period of {hold_period} candles...")

        # Filter bullish patterns for long backtests (buys)
        bullish_patterns = ['Hammer', 'Dragonfly Doji', 'Bullish Marubozu', 'Bullish Engulfing', 'Bullish Harami']
        is_bullish = selected_pattern in bullish_patterns

        # If the pattern is not bullish, a "long" backtest is not performed
        if not is_bullish:
             print("This lightweight backtest only simulates long positions (buys). Please select a bullish pattern.")
             return

        # Backtest
        results = backtest_patron(df.copy(), patterns_df, selected_pattern, hold_period)

        # Show results
        if results['total_trades'] > 0:
            win_rate = (results['winning_trades'] / results['total_trades']) * 100
            print("\n### Backtest Results:")
            print(f"Pattern analyzed: {selected_pattern}")
            print(f"Total trades: {results['total_trades']}")
            print(f"Win Rate: {win_rate:.2f}%")
            print(f"Total Return (ROI): {(results['total_return'] * 100):.2f}%")
            print(f"Maximum Drawdown: {(results['max_drawdown'] * 100):.2f}%")
        else:
            print("No trades were found for this pattern. Please check your selections.")

# Create the backtest widgets
pattern_dropdown_options = list(patterns_df['pattern'].unique()) if 'patterns_df' in globals() and not patterns_df.empty else ['No patterns detected']
pattern_dropdown = widgets.Dropdown(
    options=pattern_dropdown_options,
    description='Pattern to Backtest:',
    disabled=('patterns_df' not in globals() or patterns_df.empty),
    style=style
)
hold_period_input = widgets.IntText(
    value=5,
    description='Holding Period (candles):',
    min=1,
    style=style
)
backtest_button = widgets.Button(
    description='Run Backtest',
    button_style='success',
    icon='play'
)

backtest_button.on_click(run_backtest)

print("Select a bullish pattern and holding period to simulate historical performance.")
display(pattern_dropdown, hold_period_input, backtest_button, backtest_output)


# In[7]:
### 7. Pattern Reliability Statistics
#
# Measures the empirical effectiveness of candlestick patterns in your dataset.
# You can define a target percentage change and the number of candles needed to reach it.

def calculate_reliability(df_original, patterns_df, percentage_change, candles_ahead):
    """
    Calculates the reliability of patterns for a given percentage change and time period.
    """
    reliability_results = {}

    # Lists of bullish and bearish patterns to know the expected direction
    bullish_patterns = ['Hammer', 'Dragonfly Doji', 'Bullish Marubozu', 'Bullish Engulfing', 'Bullish Harami']
    bearish_patterns = ['Shooting Star', 'Gravestone Doji', 'Bearish Marubozu', 'Bearish Engulfing', 'Bearish Harami']

    unique_patterns = patterns_df['pattern'].unique()

    for pattern in unique_patterns:
        # Ignore indecision patterns for this direction metric
        if pattern not in bullish_patterns and pattern not in bearish_patterns:
            continue

        detections = patterns_df[patterns_df['pattern'] == pattern]
        total_detecciones = len(detections)
        successful_detections = 0

        if total_detecciones == 0:
            continue

        is_bullish = pattern in bullish_patterns

        for date_found in detections['date']:
            try:
                # Find the index of the detection candle
                current_idx = df_original.index.get_loc(date_found)

                # Ensure there are enough candles for the analysis period
                if current_idx + candles_ahead < len(df_original):
                    current_price = df_original.iloc[current_idx]['Close']
                    future_price = df_original.iloc[current_idx + candles_ahead]['Close']

                    change = (future_price - current_price) / current_price

                    # Check for success according to the pattern's direction
                    if is_bullish and change >= percentage_change / 100:
                        successful_detections += 1
                    elif not is_bullish and change <= -percentage_change / 100:
                        successful_detections += 1

            except KeyError:
                continue

        # Save the results
        success_rate = (successful_detections / total_detecciones) * 100 if total_detecciones > 0 else 0
        reliability_results[pattern] = {
            'Total Detections': total_detecciones,
            'Successful Detections': successful_detections,
            'Reliability Rate (%)': round(success_rate, 2)
        }

    return pd.DataFrame.from_dict(reliability_results, orient='index')

# Widgets for reliability statistics
stats_output = widgets.Output()

def run_stats(b):
    with stats_output:
        clear_output(wait=True)
        if 'df' not in globals() or df.empty or 'patterns_df' not in globals() or patterns_df.empty:
            print("Please download the data and detect patterns first in the previous sections.")
            return

        percentage_change = change_percentage_input.value
        candles_ahead = periods_input.value

        if percentage_change <= 0 or candles_ahead <= 0:
            print("The percentage change and number of candles must be greater than zero.")
            return

        print(f"Calculating reliability for a {percentage_change}% change in {candles_ahead} candles...")

        results = calculate_reliability(df.copy(), patterns_df.copy(), percentage_change, candles_ahead)

        if results.empty:
            print("No patterns found or reliability could not be calculated.")
        else:
            print("\n### Pattern Reliability Results:")
            display(results.sort_index())

# Create the reliability widgets
change_percentage_input = widgets.FloatText(
    value=2.0,
    description='Percentage Change (%):',
    min=0.1,
    style=style
)
periods_input = widgets.IntText(
    value=10,
    description='Period (candles):',
    min=1,
    style=style
)
calculate_button = widgets.Button(
    description='Calculate Reliability',
    button_style='info',
    icon='check'
)

calculate_button.on_click(run_stats)

print("Define the parameters to measure the reliability of the patterns:")
display(change_percentage_input, periods_input, calculate_button, stats_output)


# ### Context and Interpretation of Patterns

# - **Doji**: A pattern of indecision, where buying and selling pressure are balanced. It often precedes a market direction change.
# - **Hammer**: A bullish reversal pattern. It indicates that the asset was sold aggressively, but buyers managed to push the price back up.
# - **Shooting Star**: A bearish reversal pattern. It suggests that buyers tried to push the price up, but sellers took control and pushed it back down.
# - **Bullish Engulfing**: A strong bullish reversal pattern. A large bullish candle completely engulfs the previous bearish candle, showing a shift in momentum.
# - **Bearish Engulfing**: A strong bearish reversal pattern. A large bearish candle completely engulfs the previous bullish candle, showing that sellers have taken control.
# - **Bullish Harami**: A bullish reversal pattern. A large bearish candle is followed by a small bullish candle contained within its body, suggesting a potential loss of bearish momentum.
# - **Bearish Harami**: A bearish reversal pattern. A large bullish candle is followed by a small bearish candle contained within its body, suggesting a potential loss of bullish momentum.
# - **Spinning Top**: Another indecision pattern, similar to the Doji but with a slightly larger body. It indicates that neither buyers nor sellers are in control.
# - **Dragonfly Doji**: Indicates a strong bullish reversal, as the price was pushed down but buyers brought it back to the open/close price.
# - **Gravestone Doji**: Indicates a strong bearish reversal, as buyers failed to maintain the high price and sellers pushed it back to the open/close price.
# - **Marubozu**: Indicates a strong movement. A bullish Marubozu suggests total control by buyers, while a bearish Marubozu suggests total control by sellers.


HTML(value='\n<style>\n    .header-container {\n        display: flex;\n        align-items: center;\n        …

Enter the ticker and date range:


Text(value='BTC-USD', description='Asset Ticker:', placeholder='Ex: AAPL, TSLA, BTC-USD', style=DescriptionSty…

DatePicker(value=datetime.date(2025, 3, 23), description='Start Date:', style=DescriptionStyle(description_wid…

DatePicker(value=datetime.date(2025, 9, 19), description='End Date:', style=DescriptionStyle(description_width…

...

Define the parameters to measure the reliability of the patterns:


FloatText(value=2.0, description='Percentage Change (%):', style=DescriptionStyle(description_width='initial')…

IntText(value=10, description='Period (candles):', style=DescriptionStyle(description_width='initial'))

Button(button_style='info', description='Calculate Reliability', icon='check', style=ButtonStyle())

Output()